In [1]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore') # 경고 문구 생략

# 데이터 경로
data_path = 'data/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 피처명 한글화

In [2]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가(루블)',
                                          'item_cnt_day': '일_판매량'})
shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})
items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})
item_categories = item_categories.rename(columns=
                                         {'item_category_name': '상품분류명',
                                          'item_category_id': '상품분류ID'})
test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

### 데이터 다운캐스팅

In [3]:
def downcast(df):
    before_me = df.memory_usage().sum() / 1024
    for col in df.columns:
        dtype_name = df[col].dtype.name
        # 오브젝트 타입이면 패스
        if dtype_name == 'object':
            pass
        # bool 타입이면 int8
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int'):
            # to_numeric() -> 숫자로 바꿔줌
            # downcast -> 알맞게 사이즈 변경
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    after_me = df.memory_usage().sum() / 1024
    print('{:.2f}% 압축!'.format(100 * (before_me - after_me) / before_me))
    
    return df

In [4]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

50.00% 압축!
38.60% 압축!
54.15% 압축!
39.95% 압축!
70.83% 압축!


### 데이터 조합 생성

In [5]:
sales_train['월ID'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
      dtype=int8)

In [6]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가(루블),일_판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [7]:
# 판매가가 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['판매가(루블)'] > 0]
# 판매가가 50,000보다 작은 데이터 추출
sales_train = sales_train[sales_train['판매가(루블)'] < 50000]

# 일_판매량이 0보다 큰 데이터 추출
sales_train = sales_train[sales_train['일_판매량'] > 0]
# 일_판매량이 1,500보다 작은 데이터 추출
sales_train = sales_train[sales_train['일_판매량'] < 1500]

In [8]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가(루블),일_판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0


In [9]:
sales_train.describe().T

,count,mean,std,min,25%,50%,75%,max
월ID,2928488.0,14.569761,9.422951,0.00,7.0,14.0,23.0,33.0
상점ID,2928488.0,33.002977,16.225425,0.00,22.0,31.0,47.0,59.0
상품ID,2928488.0,10200.281556,6324.399467,0.00,4477.0,9355.0,15691.0,22169.0
판매가(루블),2928488.0,889.325169,1717.566349,0.07,249.0,399.0,999.0,49782.0
일_판매량,2928488.0,1.247597,2.285180,1.00,1.0,1.0,1.0,1000.0


In [10]:
shops['도시명'] = shops['상점명'].apply(lambda x: x.split()[0])

In [11]:
shops.head()

,상점명,상점ID,도시명
0,"!Якутск Орджоникидзе, 56 фран",0,!Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,!Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [12]:
print(shops['도시명'].unique())
shops['도시명'].nunique()

['!Якутск' 'Адыгея' 'Балашиха' 'Волжский' 'Вологда' 'Воронеж' 'Выездная'
 'Жуковский' 'Интернет-магазин' 'Казань' 'Калуга' 'Коломна' 'Красноярск'
 'Курск' 'Москва' 'Мытищи' 'Н.Новгород' 'Новосибирск' 'Омск'
 'РостовНаДону' 'СПб' 'Самара' 'Сергиев' 'Сургут' 'Томск' 'Тюмень' 'Уфа'
 'Химки' 'Цифровой' 'Чехов' 'Якутск' 'Ярославль']


32

In [13]:
# 오류 제거
shops.loc[shops['도시명'] =='!Якутск', '도시명'] = 'Якутск'

In [14]:
shops.head()

,상점명,상점ID,도시명
0,"!Якутск Орджоникидзе, 56 фран",0,Якутск
1,"!Якутск ТЦ ""Центральный"" фран",1,Якутск
2,"Адыгея ТЦ ""Мега""",2,Адыгея
3,"Балашиха ТРК ""Октябрь-Киномир""",3,Балашиха
4,"Волжский ТЦ ""Волга Молл""",4,Волжский


In [15]:
from sklearn.preprocessing import LabelEncoder

# 레이블 인코더 생성
label_encoder = LabelEncoder()
# 도시명 피처 레이블 인코딩
shops['도시명'] = label_encoder.fit_transform(shops['도시명'])

In [16]:
# 상점 피처 제거
shops = shops.drop('상점명', axis=1)
shops.head()

,상점ID,도시명
0,0,29
1,1,29
2,2,0
3,3,1
4,4,2


### 대분류

In [17]:
item_categories.head()

,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [18]:
item_categories['대분류'] = item_categories['상품분류명'].apply(lambda x: x.split()[0])  
item_categories['대분류'].value_counts()

Игры          14
Книги         13
Подарки       12
Игровые        8
Аксессуары     7
Музыка         6
Программы      6
Карты          5
Кино           5
Служебные      2
Чистые         2
PC             1
Билеты         1
Доставка       1
Элементы       1
Name: 대분류, dtype: int64

In [19]:
item_categories[item_categories['대분류']=='Игры']

,상품분류명,상품분류ID,대분류
18,Игры - PS2,18,Игры
19,Игры - PS3,19,Игры
20,Игры - PS4,20,Игры
21,Игры - PSP,21,Игры
22,Игры - PSVita,22,Игры
23,Игры - XBOX 360,23,Игры
24,Игры - XBOX ONE,24,Игры
25,Игры - Аксессуары для игр,25,Игры
26,Игры Android - Цифра,26,Игры
27,Игры MAC - Цифра,27,Игры


In [20]:
def replace_etc(word):
    if len(item_categories[item_categories['대분류']==word]) < 5:
        return 'etc'
    else:
        return word

In [21]:
# 대분류의 고윳값 개수 5개 미만 'etc'
item_categories['대분류'] = item_categories['대분류'].apply(replace_etc)

In [22]:
item_categories.head()

,상품분류명,상품분류ID,대분류
0,PC - Гарнитуры/Наушники,0,etc
1,Аксессуары - PS2,1,Аксессуары
2,Аксессуары - PS3,2,Аксессуары
3,Аксессуары - PS4,3,Аксессуары
4,Аксессуары - PSP,4,Аксессуары


In [23]:
item_categories['대분류'] = label_encoder.fit_transform(item_categories['대분류'])

In [24]:
from itertools import product

# 월ID, 상점ID, 상품ID 프레임 생성 
train = []
for i in sales_train['월ID'].unique():
    about_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    about_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    # itertools의 product는 데카르트 곱으로 A집합의 x이면서 B집합에서는 y인 순서쌍들의 집합
    train.append(np.array(list(product([i], about_shop, about_item))))
# 피처 설정
features = ['월ID', '상점ID', '상품ID']
# 리스트 타입인 train을 DataFrame 타입으로 변환 
train = pd.DataFrame(np.vstack(train), columns=features)
train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10884455,33,21,7635
10884456,33,21,7638
10884457,33,21,7640
10884458,33,21,7632


### 타깃값(월간 판매량) 추가

In [25]:
sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가(루블),일_판매량
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0


In [26]:
# # features 그룹화해 판매량 합 구하기 
# target = sales_train.groupby(features).agg({'일_판매량': 'sum',
#                                            '판매가(루블)':'mean'})
# target = target.reset_index()
# target = target.rename(columns={'일_판매량': '월간 판매량','판매가(루블)': '평균 판매가'})

In [27]:

# features 그룹화해 판매량 합 구하기 
target = sales_train.groupby(features).agg({'일_판매량': 'sum'})
target = target.reset_index()
target = target.rename(columns={'일_판매량': '월간 판매량'})

In [28]:
target

,월ID,상점ID,상품ID,월간 판매량
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1608219,33,59,22087,6.0
1608220,33,59,22088,2.0
1608221,33,59,22091,1.0
1608222,33,59,22100,1.0


In [29]:
# train과 target 병합
train = train.merge(target, on=features, how='left')
del target

In [30]:
train

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10884455,33,21,7635,NaN
10884456,33,21,7638,NaN
10884457,33,21,7640,NaN
10884458,33,21,7632,NaN


In [31]:
# target = sales_train.groupby(features).agg({'일_판매량': 'count'})
# target = target.reset_index()
# target = target.rename(columns={'일_판매량': '판매건수'})
# train = train.merge(target, on=features, how='left')
# train.head()

### 테스트 데이터 이어붙이기

In [32]:
test['월ID'] = 34
# 불필요한 ID 제거
total_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True,
                     keys=features)

In [33]:
# 결측값 0 입력
total_data = total_data.fillna(0)

total_data.head()

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0


### 최종 데이터 생성

In [34]:
# 공통 항목을 기준으로 merge() 사용
total_data = total_data.merge(shops, on='상점ID', how='left')
total_data = total_data.merge(items, on='상품ID', how='left')
total_data = total_data.merge(item_categories, on='상품분류ID', how='left')

# 데이터 다운캐스팅
total_data = downcast(total_data)

38.60% 압축!


In [35]:
total_data.head()

,월ID,상점ID,상품ID,월간 판매량,도시명,상품명,상품분류ID,상품분류명,대분류
0,0,59,22154,1.0,30,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,5
1,0,59,2552,0.0,30,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,7
2,0,59,2554,0.0,30,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,7
3,0,59,2555,0.0,30,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,7
4,0,59,2564,0.0,30,DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео,7


In [36]:
total_data = total_data.drop(['상품명', '상품분류명'], axis=1)

In [37]:
total_data

,월ID,상점ID,상품ID,월간 판매량,도시명,상품분류ID,대분류
0,0,59,22154,1.0,30,37,5
1,0,59,2552,0.0,30,58,7
2,0,59,2554,0.0,30,58,7
3,0,59,2555,0.0,30,56,7
4,0,59,2564,0.0,30,59,7
...,...,...,...,...,...,...,...
11098655,34,45,18454,0.0,20,55,7
11098656,34,45,16188,0.0,20,64,8
11098657,34,45,15757,0.0,20,55,7
11098658,34,45,19648,0.0,20,40,5


### 데이터 나누기

In [38]:
# 훈련 data -> 2013년 1월(0) 부터 2015년 9월(32)
X_train = total_data[total_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)
# 검증 data -> 가장 최신 데이터 2015년 10월(33)
X_valid = total_data[total_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)
# 테스트 data -> 위에서 추가한 예측 데이터 == 34
X_test = total_data[total_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# target data
y_train = total_data[total_data['월ID'] < 33]['월간 판매량']
# 0~20 제한
y_train = y_train.clip(0, 20)
# target 검증 data
y_valid = total_data[total_data['월ID'] == 33]['월간 판매량']
y_valid = y_valid.clip(0, 20)

In [39]:
# # 가비지 컬렉터
# import gc

# del target, shops, items, item_categories, total_data
# gc.collect();

### 모델 훈련 및 성능 검증

In [40]:
import lightgbm as lgb

In [41]:
# params = {
#     'boosting_type': 'gbdt',    # 부스팅 타입
#     'objective': 'regression',  # 손실 함수
#     'metric': 'mse',            # 평가 지표
#     'num_leaves': 31,           # 트리의 최대 잎(리프) 수
#     'learning_rate': 0.05,      # 학습률
#     'feature_fraction': 0.9,    # feature를 사용할 비율
#     'bagging_fraction': 0.8,    # 데이터 샘플링 비율
#     'bagging_freq': 5,          # 데이터 샘플링의 빈도
#     'max_depth': -1,            # 트리의 최대 깊이
#     'n_jobs': -1,               # CPU 사용 개수 (-1은 모든 CPU 사용)
#     'seed': 42                  # 랜덤 시드
# }

In [42]:
# Lightgbm 하이퍼파라미터 지정
params = {'metric': 'rmse', # 평가지표
          'num_leaves': 300, #  트리의 최대 잎(리프)수
          'learning_rate': 0.005, # 학습률
          'force_col_wise': True, # 열 단위 방식으로 데이터를 처리
          'feature_fraction': 0.9, # feature를 사용할 비율
          'bagging_fraction': 0.8, # 데이터 샘플링 비율
          'bagging_freq': 5,  # 데이터 샘플링의 빈도
          'random_state': 10} # 시드값 고정

# 카테고리 설정 -> 범주형 데이터 입력
# 상품ID -> 고윳값 개수 많은 범주형 데이터 -> 수치형 데이터 취급
# For a categorical feature with high cardinality (#category is large),
# it often works best to treat the feature as numeric,
# either by simply ignoring the categorical interpretation of the integers
# or by embedding the categories in a low-dimensional numeric space.

cat_features = ['상점ID', '상품분류ID','도시명','대분류']

# Lightgbm 훈련 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
# Lightgbm 검증 데이터셋
dvalid = lgb.Dataset(X_valid, y_valid)

In [43]:
X_train.head()

,월ID,상점ID,상품ID,도시명,상품분류ID,대분류
0,0,59,22154,30,37,5
1,0,59,2552,30,58,7
2,0,59,2554,30,58,7
3,0,59,2555,30,56,7
4,0,59,2564,30,59,7


In [44]:
# Lightgbm 훈련 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
# Lightgbm 검증 데이터셋
dvalid = lgb.Dataset(X_valid, y_valid)
# LightGBM 모델 훈련
lgb_model = lgb.train(params=params, #파라미터 지정
                      train_set=dtrain, # 학습 데이터셋 지정
                      num_boost_round=2000, # 부스팅 반복횟수 지정
                      valid_sets=(dtrain, dvalid), # 검증 데이터 지정
                      categorical_feature=cat_features, #카테고리 설정
                      early_stopping_rounds=200, # 조기 종료
                      verbose_eval=100) # 출력 빈도 조절

[LightGBM] [Info] Total Bins 467
[LightGBM] [Info] Number of data points in the train set: 10646376, number of used features: 6
[LightGBM] [Info] Start training from score 0.300535
Training until validation scores don't improve for 200 rounds
[100]	training's rmse: 1.15349	valid_1's rmse: 1.06801
[200]	training's rmse: 1.11761	valid_1's rmse: 1.03725
[300]	training's rmse: 1.09759	valid_1's rmse: 1.02221
[400]	training's rmse: 1.0852	valid_1's rmse: 1.01461
[500]	training's rmse: 1.07646	valid_1's rmse: 1.00977
[600]	training's rmse: 1.06909	valid_1's rmse: 1.00661
[700]	training's rmse: 1.06278	valid_1's rmse: 1.00447
[800]	training's rmse: 1.05792	valid_1's rmse: 1.00305
[900]	training's rmse: 1.05393	valid_1's rmse: 1.00192
[1000]	training's rmse: 1.04962	valid_1's rmse: 1.00159
[1100]	training's rmse: 1.04617	valid_1's rmse: 1.00148
[1200]	training's rmse: 1.04308	valid_1's rmse: 1.00123
[1300]	training's rmse: 1.04036	valid_1's rmse: 1.001
[1400]	training's rmse: 1.0379	valid_1's 

### 예측 및 결과 제출

In [45]:
# 예측
pred = lgb_model.predict(X_test).clip(0, 20)
# 제출 파일 생성
submission['item_cnt_month'] = pred
submission.to_csv('submission.csv', index=False)

In [46]:
submission

,ID,item_cnt_month
0,0,0.653666
1,1,0.356559
2,2,0.449363
3,3,0.384129
4,4,0.491137
...,...,...
214195,214195,0.123518
214196,214196,0.133995
214197,214197,0.117968
214198,214198,0.098072


In [47]:
# del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
# gc.collect();